<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2024.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>,
    <a href="mailto:zhiyuan.hu@epfl.ch">Zhiyuan Hu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 1.1: Pixel-wise operations</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday September 26th, 2024</p>
    <p style="margin:4px;"><b>Submission</b>: Monday October 7th, 2024 (before 23:59) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <!--number of points is sum of both parts of the lab -->
    <p style="margin:4px;"><b>Grade weight</b>: Lab 1 (16 points), 9% of the overall grade</p>
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 1</p>
</div>

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

### Imports
In the next cell we import the python libraries and load the images that we will use throughout the lab similar to the previous notebook. Run the next cell to get your notebook ready.
<div class="alert alert-info">
    
<b>Note:</b> While in the pixelwise operations lab we didn't need high accuracy, for transforms like the Fourier transform, as well as other transforms, it is essential to have the highest accuracy available.
</div>

In [ ]:
# Configure plotting as dynamic
%matplotlib widget

# Import required packages for this lab
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from skimage import io

from interactive_kit import imviewer as viewer

# Loading images
joux = io.imread("images/joux.tif").astype('float64')
car = io.imread("images/car_pad.tif").astype('float64')
mandrill = io.imread("images/mandrill.tif").astype('float64')
impulse = np.zeros((65,65)); impulse[32,32] = 1;
pens = io.imread("images/pens.tif").astype('float64')
zebra = io.imread("images/zebra.tif").astype('float64')

# The Fourier transform (7 points)

In this second part we will look at the 2D discrete Fourier transform (*DFT*).

You are going to:
 * understand the effects of the elements in an image on its Fourier transform (*FT*), and then
 * understand how an image is reconstructed from its FT using the inverse Fourier transform (*iFT*). 

In this part of the lab we will only use Python. To compute the FT in Python, we will use the [`fft` module](https://numpy.org/doc/stable/reference/routines.fft.html) in NumPy, which implements the *FT* using a [fast Fourier transform (FFT)](https://en.wikipedia.org/wiki/Fast_Fourier_transform) algorithm.

## <a id="ToC_2_Fourier_transform"></a>Table of contents

1. [Understanding the Fourier transform and its inverse](#1.-Understanding-the-Fourier-transform-and-its-inverse-(4-points)) **(4 points)**
2. [Reconstruction of an image](#2.-Reconstruction-of-an-image-(3-points)) **(3 points)**
    1. [Reconstruction error](#2.A.-Reconstruction-error-(1-point))
    2. [Fourier components](#2.B.-Fourier-components-(1-point))

Run the next cell to display the images and get familiar with them.

In [ ]:
image_list = [joux, car, mandrill, impulse, pens, zebra]
initial_viewer = viewer(image_list, hist=True)

# 1. Understanding the Fourier transform and its inverse (4 points)
[Back to table of contents](#ToC_2_Fourier_transform)

First, we will provide the functions `fourier(img)` and `inverse_fourier(ft)`, which calculate the FT and the iFT respectively.

Make sure to understand how we use the <a href='https://numpy.org/doc/stable/reference/routines.fft.html'><code>numpy.fft</code></a> module.

<!--
<div class="alert alert-info">
<b>Note:</b> The <a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.fft2.html'><code>ft = np.fft.fft2(img)</code></a> function computes the 2D Fourier transform <code>ft</code> of an image <code>img</code> within the frequency range $[0, \pi]$. In this representation, when displaying <code>ft</code>, the zero-frequency component is located at the origin of the image (top left corner). However, it's more natural to have the zero-frequency component at the center of the image <code>ft</code>. To achieve this, we can apply the <a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.fftshift.html#numpy.fft.fftshift'><code>np.fft.fftshift(ft)</code></a> function, which shifts the frequency range of <code>ft</code> from $[0, \pi]$ to $[-\frac{\pi}{2}, \frac{\pi}{2}]$. 
</div>
-->

In [ ]:
# Function that returns the FT
def fourier(img):
    # Generate the FT
    ft = np.fft.fft2(img)
    # Shift the frequency range to [-pi/2, pi/2] 
    shift_ft = np.fft.fftshift(ft)
    return shift_ft

# Function that return the inverse FT
def inverse_fourier(ft):
    # Shift the FT back to [0, pi]
    ft = np.fft.ifftshift(ft)
    # Get the inverse FT
    ift = np.fft.ifft2(ft)
    # Clip the imaginary part of the reconstruction
    # (should be approximately zero anyway)
    ift = np.real(ift)
    return ift

Calculating the FT of an image generates a two-dimensional array (image) of complex values, which makes it challenging to visualize effectively. Therefore, we usually extract the **magnitude** and **phase** of the complex numbers. The magnitude of a complex number $z\in\mathbb{C}$ is given by

$$|z| = \sqrt{\operatorname{Re}(z)^2+\operatorname{Im}(z)^2}.$$

Moreover, we usually visualize this magnitude in decibels (dB) using the formula

$$|z|~[\mathrm{dB}] = 10\log_{10}\left(|z|^2\right)= 20\log_{10}\left(|z|\right).$$

This approach is favored because the magnitude of the Fourier transform can vary significantly, covering both small and large values. The logarithmic transformation enables us to represent both ranges in a single image.

For **1 point**, complete the function `magnitude(ft)` which should return **the magnitude in decibels (`dB`)** of a FT given as an input parameter
 
<div class="alert alert-info">

<b>Hints:</b>
Check the Numpy built-in functions for complex numbers, pay attention to the base of the log function you use.
</div>

<div class="alert alert-danger">
<b>Warning:</b> Using <code>np.absolute</code> in this exercise will give you <b>0 points</b>! We want you to implement the function yourself.
</div>

In [ ]:
# Function that returns the magnitude of the FT in dB
def magnitude(ft):
    output = None
    
    # YOUR CODE HERE
    
    return output

In [ ]:
# Let's do a sanity check
# The complex number used for the test which has a magnitude of ~3 dB
z = 1 + 1j
# Check that the magnitude function is correct
if np.round(magnitude(z), decimals=1) == 3.0:
    print("Nice, your magnitude function passed the basic sanity check!")
else:
    print("Something isn't quite right yet.")


Now, we will define a function to calculate the phase of the *FT*. For this we define the function `phase(ft)`, which takes as argument an *FT* and returns its phase.

Remember that the phase of a complex number $z$ is given by
$$\angle(z)=\arctan\left(\frac{\operatorname{Im}(z)}{\operatorname{Re}(z)}\right)\,.$$

For **0.5 points**, complete the function `phase(ft)` in the cell below according to the equation given above.

<div class="alert alert-info">
<b>Hint:</b> Compare <a href="https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html" ><code>np.arctan2()</code></a> and <a href="https://numpy.org/doc/stable/reference/generated/numpy.arctan.html#numpy.arctan" ><code>np.arctan()</code></a>, which one is more convenient to calculate $\angle(z)$? 
</div>
<div class="alert alert-danger">
<b>Warning:</b> Using <code>np.angle</code> in this exercise will give you <b>0 points</b>! Implement the function yourself.
</div>

In [ ]:
# Function that calculates the phase of complex numbers
def phase(ft):
    output = None
    
    # YOUR CODE HERE
    
    return output

In [ ]:
# Let's do a sanity check
# The complex number used for the test which has a phase of pi/4
z = 1 + 1j
# Check that the magnitude function is correct
if phase(z) == np.pi/4:
    print("Nice, your phase function passed the sanity check!")
else:
    print("Something isn't quite right yet.")

Now, let's run the next cell to observe the outcomes of the functions you just coded. For this we will apply the *FT* to the image `car`, calculate its magnitude and phase, and visualize the results as images.

</div>
<div class="alert alert-success">
<b>Hint:</b> If you don't see the phase of the image, use the sliding bar to slide to the right. You can also use <code>Ctrl + b</code> to hide the left sidebar of JupyterLab. 
</div>

If everything went well you should see:
<ul><li>For the magnitude: a diagonal cross in the center with stars spread over the image, and</li>
<li>For the phase: random noise, cut by a near vertical line and a few other straight lines.</li>
</ul>


In [ ]:
# Generate the FT of car with its magnitude and phase
car_ft = fourier(car)
car_ft_mag = magnitude(car_ft)
car_ft_ph = phase(car_ft)

# Visualize 
plt.close('all')
ft_vis = viewer([car, car_ft_mag, car_ft_ph], title=['Car', 'Car FT magnitude', 'Car FT phase'], subplots=(1,3))

### Multiple Choice Questions 
The following MCQs will test your understanding of the relationship of an image with its FT **magnitude**. Each is worth **0.5 points**.

Run the next cell to visualize the images `pens` and `car` together with their FT magnitudes and answer the upcoming questions.

In [ ]:
# Get the FT magnitudes of the images using the fourier and magnitude functions
car_ft  = magnitude(fourier(car))
pens_ft = magnitude(fourier(pens))

# Define the lits of images and names
image_list = [car, car_ft, pens, pens_ft]
title_list = ['Car', 'FT of Car', 'Pens', 'FT of Pens']

# Display results
plt.close('all')
ft_viewer = viewer(image_list, title=title_list, subplots=(2,2), colorbar=True)

* Q1: Where do the little stars at different distances from the center in the FT of `car` come from?
    1. From the car.
    2. From the driver.
    3. From the carpet under the car.
    4. From the details of the car (JAGUAR text, doors, steering wheel, etc).
    5. From the size of the image.

In the next cell, modify the variable `answer` to reflect your choice.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
assert answer in [1, 2, 3, 4, 5], 'Possible answers are 1, 2, 3, 4 and 5'

* Q2: Where do the two big intersecting lines in `car` come from? 
    1. From the contour of the car.
    2. From the driver.
    3. From the carpet under the car.
    4. From the details of the car (JAGUAR text, doors, steering wheel, etc).
    5. From the size of the image.

In the next cell, modify the variable `answer` to reflect your choice.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
assert answer in [1, 2, 3, 4, 5], 'Valid answers are 1, 2, 3, 4, and 5'

* Q3: Why is there only one main line in the FT of `pens`, if there are two pens?
    1. Because the background is constant.
    2. Because they are ballpoint pens and not fountain pens.
    3. Because the two pens are aligned.
    4. Because the two pens are close to each other.

In the next cell, modify the variable `answer` to reflect your choice.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
assert answer in [1, 2, 3, 4], 'Valid answers are 1, 2, 3 or 4'

* Q4: Why is the main line in the *FT* not aligned with the pens? 

    1. The frequency of a contour is perpendicular to the contour. 
    2. The main periodicity in the image is *parallel* to the pens because they have rough surfaces.
    3. The `viewer` rotates the FT.
    4. Numpy rotates the FT.

In the next cell, modify the variable `answer` to reflect your choice.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
assert answer in [1, 2, 3, 4, 5], 'Valid answers are 1, 2, 3, 4 and 5'

# 2. Reconstruction of an image (3 points)
[Back to table of contents](#ToC_2_Fourier_transform)

As you learned in the course, it is possible to reconstruct an image from its Fourier transform by performing the inverse Fourier transform. In the next exercise we will investigate the role that the magnitude and phase of the *FT* has on the reconstruction of an image. For this we first need to create a function that reconstructs an image from its *FT* magnitude (in $\mathrm{dB}$) and phase.

Run the next cell to define the function `reconstruct` and make sure that you understand every line of the code.

In [ ]:
# Function that reconstructs an image from its FT magnitude (in dB) and phase
def reconstruct(mag, ph):
    # Convert magnitude from dB to linear scale
    mag = 10 ** (mag / 20)
    # Restore complex Fourier Transform using polar representation
    ft = mag * np.exp(1j * ph)
    # Reconstruct the image using the inverse Fourier function
    return inverse_fourier(ft)

Let's see if the function works. Run the cell below to reconstruct the car image from its magnitude and phase, and visualize the result. 

<div class = 'alert alert-danger'>
<b>Warning:</b> If the reconstruction is not near perfect, check again your functions <code>magnitude</code> and <code>phase</code>. You will need both functions to answer the next questions. 
</div>

In [ ]:
# Reconstruct the car image
car_reconstructed = reconstruct(car_ft_mag, car_ft_ph)
# Display the result
plt.close('all')
ft_rec_vis = viewer([car, car_reconstructed], title=['Original car', 'Reconstructed car'], subplots=(1,2))
np.testing.assert_array_almost_equal(car, car_reconstructed, err_msg='Check again your magnitude and phase functions!')

Since we didn't make any changes to the *FT* before the reconstruction, the reconstructed image should be (almost) identical to the original image (if it's not, you should have seen an error message). 

For the next excercise we will use the `mandrill` image in addition to the car image. Run the next cell to visualize it again. Moreover, we will plot its *FT*'s magnitude and phase (browse through the images with the buttons `Next` & `Prev`).

In [ ]:
# Generate FT of the mandrill image and extract the magnitude and phase
mandrill_ft = fourier(mandrill)
mandrill_ft_mag = magnitude(mandrill_ft)
mandrill_ft_ph = phase(mandrill_ft)

# Visualize
plt.close('all')
mandrill_vis = viewer([mandrill, mandrill_ft_mag, mandrill_ft_ph], widgets=True)

Now let's see what happens if we use the magnitude of one image and the phase of another image to do the reconstruction. What do you think will happen? Run the cell below and observe the results. Try to make a conclusion on what type of information is stored in the phase of the *FT*.

In [ ]:
# Reconstruct an image with the magnitude of car and phase of mandrill
car_mandrill = reconstruct(car_ft_mag, mandrill_ft_ph)
# Reconstruct an image with the magnitude of mandrill and phase of car
mandrill_car = reconstruct(mandrill_ft_mag, car_ft_ph)
# Visualize the results
plt.close('all')
rec_comp_vis = viewer([car_mandrill, mandrill_car], title=['Magn. = car, Phase = mandrill', 'Magn. = mandrill, Phase = car'], subplots=(1,2))

### Multiple choice question

What type of information of the image is stored in the **phase** of the FT that is **not stored** in the magnitude? (**0.5 points**)

1. The spacial frequencies contained in the image.
2. The light intensity of each pixel.
3. The location and shape of objects in the image.

Modify the variable `answer` in the next cell to reflect your choice.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
# Sanity check
assert answer in [1, 2, 3], 'Valid answers are 1, 2 or 3'

## 2.A. Reconstruction error (1 point)
[Back to table of contents](#ToC_2_Fourier_transform)

How many Fourier coefficients do we really need to keep to still have the basic information present in an image? Do all coefficients contribute the same? In order to quantify _how good_ a certain reconstruction is, we will use a metric to assess the quality of the reconstruction.

The metric we use is the normalized mean square error (NMSE) in dB. It calculates the difference between an image $f$ of size $K\times L$ and its reconstruction $g$ normalizes by the total power of $f$.

$$\operatorname{NMSE}_f(g) =  \frac{\sum_{k=1}^{K} \sum_{l=1}^L (g[k,l] - f[k,l])^2}{\sum_{m=1}^{K} \sum_{n=1}^L f[m,n]^2}, \qquad \qquad \operatorname{NMSE}_f(g)~[\mathrm{dB}] = 10 \log_{10}\left(\operatorname{NMSE}_f(g)\right).$$

This makes it easier for one to observe, for example, when the error has doubled ($+3~\mathrm{dB}$) or halved ($-3~\mathrm{dB}$) in plots.

For **1 point**, complete the function `nmse(f, g)` in the cell below according to the equation given above, where `f` and `g` are two NumPy arrays of the same shape.

<div class="alert alert-danger">
<b>Warning</b>: Remember that your function should <b>not use <code>for</code> loops</b> to iterate through images (this will give you $0$ points), and should work for NumPy arrays of any shape.
</div>

In [ ]:
# Function that calculates the Normalized Mean Square Error in dB
def nmse(f, g):
    output = None
    
    # YOUR CODE HERE
    
    return output

In [ ]:
# Sanity check (do not worry about the divide by zero note)
if  nmse(impulse, impulse) != -np.infty: 
    print('The error between two equal images should be zero. In dB -infinity.')
# Check your function on the hrct image
elif nmse(impulse, 0) != 0:
    print('The error of any image and a zero-image should be 1. In dB, 0.')
else:
    print('Nice, your function seems to work! Do not worry about the divide by zero warning!')

## 2.B. Fourier components (1 point)
[Back to table of contents](#ToC_2_Fourier_transform)

In this section, we look into the reconstruction process of an image from part of its Fourier components. This touches a topic that will continue to appear in IP1 and IP2: how much does a given transform compress an image? 

We define a function `clip_fourier(img, percent)` that reconstructs an image for only `percent`$\%$ of its Fourier coefficients. If `largest=True`, only the largest are kept, while 
if `largest=False`, they are excluded and only all the rest are kept. This will illustrate the uneven distribution of information contained in the Fourier components.

Run the next cell to define the function `clip_fourier`.

In [ ]:
 def clip_fourier(img, percent, largest=True, perc=True):
    # Get number of coefficients to keep
    if perc:
        n = np.round(img.size * percent / 100 ).astype(int)
    else: 
        n = percent
    img_ft = np.fft.fft2(img)
    # Get the threshold value. To do this, we order the Fourier coefficients 
    # from low to high and select the n-to-last ([-n]) coefficients
    threshold = np.sort(np.abs(img_ft.flatten()))[-n]
    # Get the inverse Fourier transform of the thresholded Fourier transform
    if largest == True:
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) >= threshold) * img_ft))
    else:
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) < threshold) * img_ft))
    return clipped_ift

Let's use the error metric `nmse` defined before to illustrate the difference in information contained in the few largest Fourier components compared to the information contained in the rest. In the cell below we will reconstruct the image `zebra` using: 
 * only the `percent` largest Fourier components, and 
 * using the `100-percent` smallest components. 

Then we will compare the reconstruction error by applying the `nmse` function defined above with both reconstructions. Run the cell below to see the different reconstruction errors. Play with the variable `percent` and see what happens.

In [ ]:
import warnings
# Suppress traitlets deprecation warning - do not modify
warnings.simplefilter("ignore")

percent = 20
# First, reconstruct zebra using the largest components
zebra_largest = clip_fourier(zebra, percent)
# Reconstruct zebra using the smallest components
zebra_smallest = clip_fourier(zebra, percent, largest=False)
# Calculate the errors
error_l = nmse(zebra,zebra_largest )
error_s = nmse(zebra,zebra_smallest)
# Compare the error
print(f'The reconstruction error when using the {percent}% largest  components: NMSE = {error_l:.4f}')
print(f"The reconstruction error when using the {100 - percent}% smallest components: NMSE = {error_s:6.4f}")
# Visualize images
view = viewer([zebra, zebra_largest, zebra_smallest], 
              title=['Original', f'{percent}% Largest Components', f'{100-percent}% Smallest Components'], 
              widgets=True)

In the long run, this type of characteristics of transforms are explored using graphs like the one below, where the NMSE can be seen as a function of the percentage of the largest coefficients kept.

In [ ]:
plt.close("all")
# Create figure
fig = plt.figure(num=f"SCIPER: {uid}",figsize=(8,5));
# Plot the NMSE vs kept coefficients curve
plt.plot(np.arange(.5, 100, 5), [nmse(zebra, clip_fourier(zebra, perc)) for perc in np.arange(.5, 100, 5)], 'bo-');
# Labels and titles for clear plotting
plt.xlabel("Percentage of coefficients kept"); plt.ylabel("NMSE [dB]"); 
plt.xticks([0, 20, 40, 60, 80, 100], [f"{perc}%" for perc in [0, 20, 40, 60, 80, 100]]);
plt.title("Reconstructing zebra with the largest Fourier coefs.")
plt.show()

Now we will create a widget to apply the function to an image and dynamically visualize its effect. 

We will define a slider to choose an integer `n` such that **the number of largest coefficients kept is $2^n$**. 
<div class = "alert alert-info">
<b>Note</b>: This is because the visual difference between the reconstructions is only apparent for percentages between $0\%$ and $2\%$, and very small steps would be needed. Keep in mind that you are not working with percentages anymore.
    
</div>

We will also provide a checkbox to switch between the two modes of operation (keeping the largest or keeping all the rest). Click the button `Apply` to apply `clip_fourier()` with the chosen parameter on the image. Run the next cell and click on `Extra Widgets` to use the widget. Explore the results carefully.

In [ ]:
# Declare slider and checkbox
n_slider = widgets.IntSlider(value=16, min=0, max=16, step=1, description='n' )
checkbox = widgets.Checkbox(value=True, description='Use largest components')

# Declare button
button = widgets.Button(description='Apply')

# Declare the button callback
def button_callback(image):
    n      = n_slider.value
    check  = checkbox.value
    output = clip_fourier(image, 2**n, largest=check, perc=False)
    return output

# Visualize
plt.close('all')
cfourier_viewer = viewer(zebra, title="Clipping the FT", new_widgets=[n_slider, checkbox, button], callbacks=[button_callback], widgets=True, normalize=True)

### Multiple Choice Question
Congratulations! You made it to the end of the notebook. Now you just need to answer these last two MCQ questions (**0.5 points** each).

* Q1: How many largest Fourier coefficients are required to start clearly seeing a zebra shape in the image?
    1. $2^{1}$
    2. $2^{4}$
    3. $2^{11}$
    4. $2^{16}$

Modify the variable `answer` to reflect your choice. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
assert answer in [1, 2, 3, 4], 'Possible answers are 1, 2, 3 and 4'

* Q2: How is it possible to reconstruct a non-periodic object such as the zebra (there is only one zebra in the image) in an image from only periodic components?
    1. The black and white stripes in the zebra make it possible.
    2. The FT assumes that the image is periodic in space.
    3. The biggest components of the FT are non-periodic, to account for such features in an image.

Modify the variable `answer` to reflect your choice. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
assert answer in [1, 2, 3], 'Possible answers are 1, 2, and 3'

<div class="alert alert-success">
<p><b>Congratulations on finishing the second part of the Pixel-Fourier lab!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1303112">Moodle</a>, in a zip file with the other notebook of this lab.
</p>
</div>

* Keep the name of the notebook as: *2_Fourier_Transform.ipynb*,
* Name the zip file: *Pixel_Fourier_Lab.zip*.